# SVM

In [13]:
#important libraries
import pandas as pd
import numpy as np
import nltk
import re
from textblob import TextBlob

#importing stopwords is optional, in this case it decreased accuracy
#from nltk.corpus import stopwords
import itertools
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.model_selection import train_test_split
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()








In [7]:
start_time = time.time()

dataset = pd.read_csv('dataset/TrumpvsBiden.csv')
dataset.head()


,author,date,text
0,R0nnieBergman,2020-10-31 09:24:34,RT @standardnews: I created The Thick of It an...
1,Godhelpusameric,2020-10-31 09:24:34,RT @realDonaldTrump: “A Biden win would mean t...
2,Zehub,2020-10-31 09:24:34,VIDEO. Election présidentielle aux États-Unis ...
3,JamieParrigin,2020-10-31 09:24:34,RT @realDonaldTrump: President Obama used to s...
4,Powerfader54,2020-10-31 09:24:34,Tucker Carlson: Hunter Biden scandal shows med...


In [8]:
def find_pol(review):
    return TextBlob(review).sentiment.polarity

#Sentiment Polarity on Trump’s dataset

dataset['Sentiment_Polarity'] = dataset['text'].apply(find_pol)
dataset.tail()

,author,date,text,Sentiment_Polarity
4249,cccelsr,2020-10-31 09:27:03,RT @themarketswork: Whom among us hasn't rando...,-0.166667
4250,UnEspanolVOX,2020-10-31 09:27:03,RT @alonso_dm: La ultraderecha cada día más ec...,0.000000
4251,hub53848550,2020-10-31 09:27:03,RT @EricTrump: TeamTrump is just getting warme...,0.000000
4252,coop22089074,2020-10-31 09:27:03,Biden Vows to Hold China Accountable amid Scru...,0.000000
4253,zeusFanHouse,2020-10-31 09:27:03,RT @MagaGlam: @realDonaldTrump We ❤️ You Presi...,0.000000


In [9]:
#Add one more attribute for Expression Label
dataset['Expression Label'] = np.where(dataset['Sentiment_Polarity']>0,'positive', 'negative')
dataset['Expression Label'][dataset.Sentiment_Polarity ==0] = "Neutral"
dataset.tail()

C:\Users\Avinash\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,author,date,text,Sentiment_Polarity,Expression Label
4249,cccelsr,2020-10-31 09:27:03,RT @themarketswork: Whom among us hasn't rando...,-0.166667,negative
4250,UnEspanolVOX,2020-10-31 09:27:03,RT @alonso_dm: La ultraderecha cada día más ec...,0.000000,Neutral
4251,hub53848550,2020-10-31 09:27:03,RT @EricTrump: TeamTrump is just getting warme...,0.000000,Neutral
4252,coop22089074,2020-10-31 09:27:03,Biden Vows to Hold China Accountable amid Scru...,0.000000,Neutral
4253,zeusFanHouse,2020-10-31 09:27:03,RT @MagaGlam: @realDonaldTrump We ❤️ You Presi...,0.000000,Neutral


In [10]:
reviews1 = dataset[dataset['Sentiment_Polarity'] == 0.0000]
reviews1.shape

(1991, 5)

In [11]:
cond1=dataset['Sentiment_Polarity'].isin(reviews1['Sentiment_Polarity'])
dataset.drop(dataset[cond1].index, inplace = True)
dataset.shape

(2263, 5)

In [12]:
dataset.head()

,author,date,text,Sentiment_Polarity,Expression Label
0,R0nnieBergman,2020-10-31 09:24:34,RT @standardnews: I created The Thick of It an...,-0.30000,negative
1,Godhelpusameric,2020-10-31 09:24:34,RT @realDonaldTrump: “A Biden win would mean t...,0.24375,positive
3,JamieParrigin,2020-10-31 09:24:34,RT @realDonaldTrump: President Obama used to s...,0.20000,positive
4,Powerfader54,2020-10-31 09:24:34,Tucker Carlson: Hunter Biden scandal shows med...,0.25000,positive
5,America_Mine,2020-10-31 09:24:34,RT @mabomm1: The Russian hoax President Trump’...,-0.25000,negative


In [14]:


labelencoder = LabelEncoder()
dataset['Expression label'] = labelencoder.fit_transform(dataset['Expression Label'])
dataset.head()

,author,date,text,Sentiment_Polarity,Expression Label,Expression label
0,R0nnieBergman,2020-10-31 09:24:34,RT @standardnews: I created The Thick of It an...,-0.30000,negative,0
1,Godhelpusameric,2020-10-31 09:24:34,RT @realDonaldTrump: “A Biden win would mean t...,0.24375,positive,1
3,JamieParrigin,2020-10-31 09:24:34,RT @realDonaldTrump: President Obama used to s...,0.20000,positive,1
4,Powerfader54,2020-10-31 09:24:34,Tucker Carlson: Hunter Biden scandal shows med...,0.25000,positive,1
5,America_Mine,2020-10-31 09:24:34,RT @mabomm1: The Russian hoax President Trump’...,-0.25000,negative,0


In [28]:
#comprehensive cleaning
def preprocess(tweet):
    
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    
    #Convert @username to __USERHANDLE
    tweet = re.sub('@[^\s]+','__USERHANDLE',tweet)  
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    
    #trim
    tweet = tweet.strip('\'"')
    
    # Repeating words like hellloooo
    repeat_char = re.compile(r"(.)\1{1,}", re.IGNORECASE)
    tweet = repeat_char.sub(r"\1\1", tweet)
    
    #Emoticons
    emoticons = \
    [
     ('__positive__',[ ':-)', ':)', '(:', '(-:', \
                       ':-D', ':D', 'X-D', 'XD', 'xD', \
                       '<3', ':\*', ';-)', ';)', ';-D', ';D', '(;', '(-;', ] ),\
     ('__negative__', [':-(', ':(', '(:', '(-:', ':,(',\
                       ':\'(', ':"(', ':((','D:' ] ),\
    ]

    def replace_parenthesis(arr):
       return [text.replace(')', '[)}\]]').replace('(', '[({\[]') for text in arr]
    
    def join_parenthesis(arr):
        return '(' + '|'.join( arr ) + ')'

    emoticons_regex = [ (repl, re.compile(join_parenthesis(replace_parenthesis(regx))) ) \
            for (repl, regx) in emoticons ]
    
    for (repl, regx) in emoticons_regex :
        tweet = re.sub(regx, ' '+repl+' ', tweet)

     #Convert to lower case
    tweet = tweet.lower()
    
    return tweet



In [29]:
#Stemming of Tweets

def stem(tweet):
        stemmer = nltk.stem.PorterStemmer()
        tweet_stem = ''
        words = [word if(word[0:2]=='__') else word.lower() \
                    for word in tweet.split() \
                    if len(word) >= 3]
        words = [stemmer.stem(w) for w in words] 
        tweet_stem = ' '.join(words)
        return tweet_stem


In [30]:
X=dataset.iloc[:,2].values
X=pd.Series(X)
y=dataset.iloc[:,5].values

        
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=5)

X_train = [stem(preprocess(tweet)) for tweet in X_train]
X_test = [stem(preprocess(tweet)) for tweet in X_test]

TypeError: expected string or bytes-like object

In [22]:
X=dataset.iloc[:,2].values
X=pd.Series(X)
y=dataset.iloc[:,5].values

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=0)

X_train = [stem(cleaning(tweet)) for tweet in X_train]
X_test = [stem(cleaning(tweet)) for tweet in X_test]


AttributeError: 'list' object has no attribute 'split'

In [ ]:
model = svm.SVC(kernel='linear') 
model.fit(train_vectors, y_train) 
predicted_sentiment = model.predict(test_vectors)


In [ ]:
print(classification_report(y_test, predicted_sentiment))

In [ ]:
predicted_sentiments = []
for s in range(len(predicted_sentiment)):
    predicted_sentiments.append(predicted_sentiment[s])
    
prediction_df = pd.DataFrame({'Content':x_test, 'Emotion_predicted':predicted_sentiment, 'Emotion_actual': y_test})
prediction_df.to_csv('emotion_recognizer_svm.csv', index = False)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")